# •	Trainer API:- 
Transformers provides a Trainer API to easily train or finetune transformer models.

In [8]:
# Here how we can easily preprocess the GLUE MRPC dataset using dynamic padding

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

In [2]:
raw_dataset = load_dataset("glue","mrpc")

Using the latest cached version of the dataset since glue couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'mrpc' at C:\Users\rajkr\.cache\huggingface\datasets\glue\mrpc\0.0.0\bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c (last modified on Wed Oct  9 14:44:28 2024).


In [5]:
checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [6]:
def tokenize_function(ex):
    return tokenizer(ex['sentence1'],ex['sentence2'],truncation=True)

In [7]:
tokenized_dataset = raw_dataset.map(tokenize_function,batched=True)
data_collator = DataCollatorWithPadding(tokenizer)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [12]:
# we also need a model and some training arguments before creating the trainer

In [9]:
from transformers import AutoModelForSequenceClassification

In [10]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
!pip install accelerate -U

In [13]:
from transformers import TrainingArguments
training_args = TrainingArguments("test-trainer",
                                 per_device_eval_batch_size=16,
                                 per_gpu_eval_batch_size=16,
                                 num_train_epochs=5,
                                 learning_rate=2e-5,
                                 weight_decay=0.01)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [15]:
# we can pass everything to the traier class and strat training


In [16]:
from transformers import Trainer

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer)
#now train the Trainer
trainer.train()

In [17]:
#The predict method allows us to get the predictions of our model on a wholel dataset
# we can then use those predictions to compute metrics

In [ ]:
predictions = trainer.predict(tokenized_dataset["validation"])
print(predictions.predictions.shape,predictions.label_ids.shape)

In [18]:
#for labels and predictions 

In [2]:
import numpy as np
from datasets import load_metric

In [3]:
metric = load_metric("glue","mrpc")
preds = np.argmax(predictions.predictions,axis=-1)
metric.compute(predictions=preds,references=predictions.label_ids)

C:\Users\rajkr\AppData\Local\Temp\ipykernel_2596\2462064265.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue","mrpc")
C:\Users\rajkr\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\load.py:753: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


NameError: name 'predictions' is not defined

# To monitor metrics during training we need to define a compute metrics function and pass it to the Trainer

In [1]:
metric = load_metric("glue","mrpc")

NameError: name 'load_metric' is not defined

In [21]:
def compute_metrics(eval_preds):
    logits,labels = eval_preds
    predictions = np.argmax(logits,axis=-1)
    return metric.compute(predictions=predictions,references=labels)

In [ ]:
training_args  = TrainingArguments("test-trainer",evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()
#now we can launch a new training with metric reporting